In [44]:
import os

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [45]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field, NonNegativeInt
from typing import List
from random import sample
from langchain.schema import HumanMessage

First, let's create a loader and load reviews from tv-reviews.csv into memory

In [46]:
from langchain.document_loaders.csv_loader import CSVLoader

loader = CSVLoader(file_path="./tv-reviews.csv")
data = loader.load()
data

[Document(metadata={'source': './tv-reviews.csv', 'row': 0}, page_content="TV Name: Imagix Pro\nReview Title: Amazing Picture Quality\nReview Rating: 9\nReview Text: I recently purchased the Imagix Pro and I am blown away by its picture quality. The colors are vibrant and the images are crystal clear. It feels like I'm watching movies in a theater! The sound is also impressive, creating a truly immersive experience. Highly recommended!"),
 Document(metadata={'source': './tv-reviews.csv', 'row': 1}, page_content="TV Name: Imagix Pro\nReview Title: Impressive Features\nReview Rating: 8\nReview Text: The Imagix Pro is packed with impressive features that enhance my viewing experience. The smart functionality allows me to easily stream my favorite shows and movies. The remote control is user-friendly and has convenient shortcuts. The slim design is sleek and fits perfectly in my living room. The only downside is that the sound could be better, but overall, I'm satisfied."),
 Document(metad

Then, let's initialize our LLM

In [47]:
model_name = "gpt-3.5-turbo"
temperature = 0.0
llm = ChatOpenAI(
    model_name=model_name,
    temperature=temperature,
    openai_api_key=os.getenv("OPENAI_API_KEY")
)

class ReviewSentiment(BaseModel):
    positives: List[NonNegativeInt] = Field(description="index of a positive TV review starting from 0")
    negatives: List[NonNegativeInt] = Field(description="index of a negative TV review starting from 0")

parser = PydanticOutputParser(pydantic_object=ReviewSentiment)
print(parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"positives": {"description": "index of a positive TV review starting from 0", "items": {"minimum": 0, "type": "integer"}, "title": "Positives", "type": "array"}, "negatives": {"description": "index of a negative TV review starting from 0", "items": {"minimum": 0, "type": "integer"}, "title": "Negatives", "type": "array"}}, "required": ["positives", "negatives"]}
```


In [48]:
prompt = PromptTemplate(
    template="{question}\n{format_instructions}\nContext: {context}",
    input_variables=["question", "context"],
    partial_variables={"format_instructions": parser.get_format_instructions}
)

question = """
    Classify TV reveiws provided in the context into positive and negative.
    Only use the reviews provided in this context, do not make up new reviews or use any existing information you know about these TVs.
    If there are no positive or negative reviews, ouput an empty JSON array.
"""

reviews_to_classify = sample(data, 3)
context = '\n'.join(review.page_content for review in reviews_to_classify)

query = prompt.format(context=context, question=question)
print(query)


    Classify TV reveiws provided in the context into positive and negative.
    Only use the reviews provided in this context, do not make up new reviews or use any existing information you know about these TVs.
    If there are no positive or negative reviews, ouput an empty JSON array.

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"positives": {"description": "index of a positive TV review starting from 0", "items": {"minimum": 0, "type": "integer"}, "title": "Positives", "type": "array"}, "negatives": {"description": "index of a negative TV review starting from 0", "i

In [49]:
query = [HumanMessage(content=query)]
output = llm(query)
print(output.content)

result = parser.parse(output.content)
print(result)

print("Positives:\n" + "\n".join([reviews_to_classify[i].page_content for i in result.positives]))
print("Negatives:\n" + "\n".join([reviews_to_classify[i].page_content for i in result.negatives]))


{
    "positives": [0, 1, 2],
    "negatives": []
}
positives=[0, 1, 2] negatives=[]
Positives:
TV Name: VisionMax Ultra
Review Title: Easy Installation Process
Review Rating: 9
Review Text: Setting up the VisionMax Ultra was a breeze. The instructions provided were clear and concise, making the installation process quick and hassle-free. The TV also offers various mounting options, allowing me to choose the perfect placement for my viewing pleasure.
TV Name: VisionMax Ultra
Review Title: Excellent Picture Clarity
Review Rating: 9
Review Text: The picture clarity of the VisionMax Ultra is simply outstanding. The images are sharp, and the colors are vivid. Whether I'm watching movies, sports, or playing games, the visuals are always stunning. This TV takes my viewing experience to a whole new level!
TV Name: VisionMax Ultra
Review Title: Sleek and Modern
Review Rating: 9
Review Text: The VisionMax Ultra fits perfectly into my modern living room decor. The slim design and minimalistic be